<a href="https://colab.research.google.com/github/ppfenninger/Sensorimotor_Learning_Final/blob/main/Sensorimotor_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Installation
!pip install pybullet > /dev/null 2>&1
!pip install git+https://github.com/taochenshh/easyrl.git > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install git+https://github.com/ppfenninger/airobot.git > /dev/null 2>&1
# !pip install git+https://github.com/ppfenninger/Sensorimotor_Learning_Final.git > /dev/null 2>&1

In [2]:
import os
import torch
import gym
import pprint
import time
import pybullet as p
import pybullet_data as pd
import pybullet_envs
import airobot as ar
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from typing import Any
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pylab
from dataclasses import dataclass
from gym import spaces
from gym.envs.registration import registry, register
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm.notebook import tqdm
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from copy import deepcopy
from itertools import count
from easyrl.agents.ppo_agent import PPOAgent
from easyrl.utils.common import save_traj
from easyrl.configs import cfg
from easyrl.configs import set_config
from easyrl.configs.command_line import cfg_from_cmd
from easyrl.engine.ppo_engine import PPOEngine
from easyrl.models.categorical_policy import CategoricalPolicy
from easyrl.models.diag_gaussian_policy import DiagGaussianPolicy
from easyrl.models.mlp import MLP
from easyrl.models.value_net import ValueNet
from easyrl.agents.base_agent import BaseAgent
from easyrl.utils.torch_util import DictDataset
from easyrl.utils.torch_util import load_state_dict
from easyrl.utils.torch_util import load_torch_model
from easyrl.runner.nstep_runner import EpisodicRunner
from easyrl.utils.torch_util import save_model
from easyrl.utils.torch_util import action_entropy
from easyrl.utils.torch_util import action_from_dist
from easyrl.utils.torch_util import action_log_prob
from easyrl.utils.torch_util import clip_grad
from easyrl.utils.common import set_random_seed
from easyrl.utils.gym_util import make_vec_env
from easyrl.utils.common import load_from_json
from easyrl.utils.torch_util import freeze_model
from easyrl.utils.torch_util import move_to
from easyrl.utils.torch_util import torch_float
from easyrl.utils.torch_util import torch_to_np
from base64 import b64encode
from IPython import display as ipythondisplay

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/easyrl/envs/shmem_vec_env.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool: ctypes.c_bool}


In [70]:
del sys.modules["de_agent"]
del sys.modules["de_runner"]
del sys.modules["utils"]
del sys.modules["de_env"]

# install our library
!rm Sensorimotor_Learning_Final -r
!git clone -b runner-testing-2 https://github.com/ppfenninger/Sensorimotor_Learning_Final.git
import sys
sys.path.insert(0, './Sensorimotor_Learning_Final/deepexploration/')
import de_agent
from de_agent import DeepExplorationAgent
# import de_engine
import de_runner # this should work once everything compiles
from de_runner import DeepExplorationRunner
import utils
from utils import generate_demonstration_data, eval_agent, load_expert_agent, create_actor

import de_env
from de_env import URRobotPusherGym

module_name = __name__

env_name = 'URPusher-v1'
if env_name in registry.env_specs:
    del registry.env_specs[env_name]
register(
    id=env_name,
    entry_point=f'{module_name}:URRobotPusherGym',
)

Cloning into 'Sensorimotor_Learning_Final'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 253 (delta 69), reused 64 (delta 38), pack-reused 154
Receiving objects: 100% (253/253), 118.91 KiB | 1.80 MiB/s, done.
Resolving deltas: 100% (131/131), done.


# Setup

In [55]:
def set_configs(exp_name='bc'):
    set_config('ppo')
    cfg.alg.seed = 2 #seed
    cfg.alg.num_envs = 1
    cfg.alg.episode_steps = 150
    cfg.alg.max_steps = 600000
    cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    cfg.alg.env_name = 'URPusher-v1'
    cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    cfg.alg.save_dir += f'/{exp_name}'
    setattr(cfg.alg, 'diff_cfg', dict(save_dir=cfg.alg.save_dir))

    print(f'====================================')
    print(f'      Device:{cfg.alg.device}')
    print(f'====================================')

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(


In [37]:

# env = make_vec_env(cfg.alg.env_name, cfg.alg.num_envs, seed=cfg.alg.seed)

[INFO][2023-05-04 15:19:07]: Alogrithm type:<class 'easyrl.configs.ppo_config.PPOConfig'>
INFO:EasyRL:Alogrithm type:<class 'easyrl.configs.ppo_config.PPOConfig'>


      Device:cpu


# Testing

In [71]:
# Runner Tests
set_configs()
env = URRobotPusherGym()

critics = []
for index in range(4):
  ob_size = env.observation_space.shape[0]
  critic_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)
  critic = ValueNet(critic_body, in_features=64)
  critics.append(critic)

actor = create_actor(env=env)
agent = DeepExplorationAgent(actor=actor, critics=critics, env=env)
runner = DeepExplorationRunner(agent=agent, env=env)

traj = runner(time_steps=cfg.alg.episode_steps)
print(traj)

[INFO][2023-05-04 15:54:51]: Alogrithm type:<class 'easyrl.configs.ppo_config.PPOConfig'>
INFO:EasyRL:Alogrithm type:<class 'easyrl.configs.ppo_config.PPOConfig'>


      Device:cpu
entering exploration more
[tensor(-0.0383), tensor(-0.0375), tensor(-0.0381), tensor(-0.0376), tensor(-0.0373), tensor(-0.0376), tensor(-0.0374), tensor(-0.0377), tensor(-0.0379), tensor(-0.0373)]


TypeError: ignored

In [21]:
print(dir(env))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_obs_from_buf', '_save_obs', 'action_space', 'actions', 'buf_dones', 'buf_infos', 'buf_obs', 'buf_rews', 'close', 'close_extras', 'closed', 'envs', 'get_images', 'get_viewer', 'keys', 'metadata', 'num_envs', 'observation_space', 'random_actions', 'render', 'reset', 'spec', 'step', 'step_async', 'step_wait', 'unwrapped', 'viewer']
